In [1]:
from validphys.api import API
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from validphys.fkparser import load_fktable
from collections import defaultdict

In [2]:
seed = 1341351341

In [3]:
def generate_sequential_model(outputs=1, 
                   input_layer=None, 
                   nlayers=2, 
                   units=[100,100],
                   seed=seed,
                   **kwargs):
  """
  Create a tensorflow sequential model where all intermediate layers have the same size
  This function accepts an already constructed layer as the input.

  All hidden layers will have the same number of nodes for simplicity

  Arguments:
      outputs: int (default=1)
          number of output nodes (how many flavours are we training)
      input_layer: KerasTensor (default=None)
          if given, sets the input layer of the sequential model
      nlayers: int
          number of hidden layers of the network
      units: int
          number of nodes of every hidden layer in the network
      activation: str
          activation function to be used by the hidden layers (ex: 'tanh', 'sigmoid', 'linear')
  """
  if len(units) != nlayers:
      raise Exception("The length of units must match the number of layers.")
  
  if kwargs.get('kernel_initializer'):
      kernel_initializer = kwargs['kernel_initializer']
  else:
      kernel_initializer = tf.keras.initializers.HeNormal

  if kwargs.get('activation_list'):
      activation_list = kwargs['activation_list']
      if len(units) != len(activation_list):
          raise Exception("The length of the activation list must match the number of layers.")
  else:
      activation_list = ['tanh', 'tanh']

  if kwargs.get('output_func'):
      output_func = kwargs['output_func']
  else:
      output_func = 'linear'
  
  if kwargs.get('name'):
      name = kwargs['name']
  else:
      name = 'pdf'
  
  model = tf.keras.models.Sequential(name=name)
  if input_layer is not None:
      model.add(input_layer)
  for layer in range(nlayers):
      model.add(tf.keras.layers.Dense(units[layer], 
                                      activation=activation_list[layer],
                                      kernel_initializer=kernel_initializer(seed=seed - layer),
                                      ),
      )
  model.add(tf.keras.layers.Dense(outputs, 
                                  activation=output_func, 
                                  kernel_initializer=tf.keras.initializers.HeNormal(seed=seed - nlayers)
                                  ))

  return model

def compute_ntk(model, input):
  grad = []
  for x in tf.convert_to_tensor(input):
    with tf.GradientTape() as tape:
      x = tf.reshape(x, shape=(-1,1))
      #tape.watch(x)
      pred = model(x)

    # compute gradients df(x)/dtheta
    g = tape.gradient(pred, model.trainable_variables)
    # concatenate the gradients of all trainable variables,
    # not discriminating between weights and biases
    size_g = len(g)
    g_minus_out = tf.concat([tf.reshape(g[i], shape=(-1,1)) for i in range(size_g - 2)], axis=0)
    g = np.array([
      np.concatenate([g_minus_out, 
                      tf.reshape(g[-2][:,i],  shape=(-1,1)), 
                      tf.reshape(g[-1][i],  shape=(-1,1))],
                      axis=0
                      )
      for i in range(pred.shape[1])
    ])
    grad.append(g)

  grad = np.array(grad)
  ntk = tf.einsum('aikl,bjkl->ijab', grad, grad)
  return ntk


def produce_ntk_Y(ntk, start_grid_by_exp, grid_size_by_exp, fk_table_list, total_ndata, start_proc_by_exp, index):
  # Constructing ntk_Y
  sub_mats = defaultdict(list)

  for exp_name_1, alpha in start_grid_by_exp.items():
    for exp_name_2, beta in start_grid_by_exp.items():
      # Take the submatrix of the NTK in data space
      ntk_red = ntk[:, :, alpha : alpha + grid_size_by_exp[exp_name_1], beta : beta + grid_size_by_exp[exp_name_2]].numpy()
      fk_I = fk_table_list[exp_name_1]
      fk_J = fk_table_list[exp_name_2]
      start_locs = (start_proc_by_exp[exp_name_1], start_proc_by_exp[exp_name_2]) # Wrong, this should have that index in the data space, not in the grid spaces
      #print("-----------------------------")
      #print(f"{fk_I.shape} - {ntk_red.shape} - {fk_J.shape}")
      fk_ntk = np.tensordot(fk_I, ntk_red, axes=[[1,2],[0,2]])
      fk_ntk_fk = np.tensordot(fk_ntk, fk_J, axes=[[1,2],[1,2]])
      sub_mats[start_locs] = fk_ntk_fk

  result = np.zeros((total_ndata, total_ndata), dtype=np.float32)
  for locs, mat in sub_mats.items():
      xsize, ysize = mat.shape
      result[locs[0] : locs[0] + xsize, locs[1] : locs[1] + ysize] = mat

  return pd.DataFrame(result, index=index, columns=index)

In [4]:
dataset_inputs = [
  #{'dataset': 'NMC_NC_NOTFIXED_DW_EM-F2', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'NMC_NC_NOTFIXED_P_EM-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'SLAC_NC_NOTFIXED_P_DW_EM-F2', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'SLAC_NC_NOTFIXED_D_DW_EM-F2', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'BCDMS_NC_NOTFIXED_P_DW_EM-F2', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'BCDMS_NC_NOTFIXED_D_DW_EM-F2', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'CHORUS_CC_NOTFIXED_PB_DW_NU-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'CHORUS_CC_NOTFIXED_PB_DW_NB-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'NUTEV_CC_NOTFIXED_FE_DW_NU-SIGMARED', 'cfac': ['MAS'], 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'NUTEV_CC_NOTFIXED_FE_DW_NB-SIGMARED', 'cfac': ['MAS'], 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'HERA_NC_318GEV_EM-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'HERA_NC_225GEV_EP-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'HERA_NC_251GEV_EP-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'HERA_NC_300GEV_EP-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'HERA_NC_318GEV_EP-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'HERA_CC_318GEV_EM-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'HERA_CC_318GEV_EP-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'HERA_NC_318GEV_EAVG_CHARM-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'HERA_NC_318GEV_EAVG_BOTTOM-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
]

In [5]:
common_dict = dict(
    dataset_inputs=dataset_inputs,
    metadata_group="nnpdf31_process",
    use_cuts='internal',
    datacuts={'t0pdfset': '240701-02-rs-nnpdf40-baseline', 'q2min': 3.49, 'w2min': 12.5},
    theoryid=40000000,
)

In [6]:
groups_data = API.procs_data(**common_dict)
groups_index = API.groups_index(**common_dict)

In [7]:
fk_table_list = defaultdict(list)
x_grid_list = defaultdict(list)
Y = []
total_ndata_wc = 0
total_grid_size = 0
start_grid_by_exp = defaultdict(list)
grid_size_by_exp = defaultdict(list)
start_proc_by_exp = defaultdict(list)
M_list = []

for idx_proc, group_proc in enumerate(groups_data):
  for idx_exp, exp_set in enumerate(group_proc.datasets):

    fkspecs = exp_set.fkspecs
    cuts = exp_set.cuts
    ndata = exp_set.load_commondata().ndata
    fk_table = load_fktable(fkspecs[0])
    fk_table_wc = fk_table.with_cuts(cuts)
    x_grid = fk_table_wc.xgrid
    fk_table_wc_np = fk_table_wc.get_np_fktable()

    Y.append(exp_set.load_commondata().central_values.to_numpy())
    fk_table_list[exp_set.name] = fk_table_wc_np
    x_grid_list[exp_set.name] = x_grid
    start_proc_by_exp[exp_set.name] = total_ndata_wc
    start_grid_by_exp[exp_set.name] = total_grid_size
    grid_size_by_exp[exp_set.name] = x_grid.shape[0]
    total_grid_size += x_grid.shape[0]
    total_ndata_wc += ndata
    M_list.append(np.tensordot(fk_table_wc_np, fk_table_wc_np, axes=[[0], [0]]))
#print(f"Total number of points after cuts: {total_ndata_wc}")

In [8]:
# Generate NNPDF model
nnpdf = generate_sequential_model(outputs=9, nlayers=2, units=[28, 20],seed=seed, name='NNPDF', kernel_initializer=tf.keras.initializers.GlorotNormal)

In [9]:
# Create index for NTK
grid_index_array = []
grid_index_id = []
counter = 0
for set, x_grid in x_grid_list.items():
  counter += x_grid.size
  for id, x in enumerate(x_grid):
    grid_index_array.append(set)
    grid_index_id.append(id)

grid_multi_index = pd.MultiIndex.from_arrays([grid_index_array, grid_index_id], names=('dataset','id'))

x_grid_total = np.concatenate([grid for grid in x_grid_list.values()])

In [10]:
ntk = compute_ntk(nnpdf, x_grid_total)

In [11]:
ntk_y = produce_ntk_Y(ntk, start_grid_by_exp, grid_size_by_exp, fk_table_list, total_ndata_wc, start_proc_by_exp, groups_index)

In [12]:
# Experimental covariance matrix
C = API.groups_covmat_no_table(**common_dict)

try:
  pd.testing.assert_index_equal(C.index, ntk_y.index)
except AssertionError as e:
  print(e)

In [13]:
ntk_hat_y = ntk_y @ np.linalg.inv(C)

In [29]:
ndat = sum([fk.shape[0] for fk in fk_table_list.values()])
FK = np.zeros(shape=(ndat, 9, x_grid_total.size))
M_list = []

id_dat = 0
id_grid = 0
for fk in fk_table_list.values():
  size_data = fk.shape[0]
  size_grid = fk.shape[2]
  FK[id_dat : id_dat + size_data, :, id_grid : id_grid + size_grid] = fk
  id_grid += size_grid
  id_dat += size_data
  m = np.tensordot(fk, fk, axes=[[0],[0]])
  m = m.reshape((
    m.shape[1],
    m.shape[3],
    m.shape[0],
    m.shape[2],
  ))
  M_list.append(m)

M = np.tensordot(FK,FK, axes=[[0],[0]])
M = M.reshape((M.shape[1], M.shape[3], M.shape[0], M.shape[2]))

In [31]:
np.linalg.inv(M[:,:,0,0])

LinAlgError: Singular matrix

In [22]:
fk_table_list['NMC_NC_NOTFIXED_P_EM-SIGMARED'][0]

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -2.38834167e-29,  5.06631223e-24,  2.23920514e-17,
        -5.74094881e-12,  2.97984601e-06,  9.12721621e-04,
        -2.20102646e-02,  1.52191518e-01,  5.36911146e-02,
         7.96573544e-02, -6.32857354e-02,  3.82231779e-02,
        -4.00606492e-03,  4.15384066e-03,  1.96418853e-03,
         1.62453063e-03,  1.27925666e-03,  1.04851831e-03,
         8.81770918e-04,  7.59575474e-04,  6.65688917e-04,
         5.93789301e-04,  5.36347195e-04,  4.86285567e-04,
         4.75786493e-04,  3.52165137e-04,  5.03362820e-04,
         1.43459864e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -8.37035227e-29,  5.41993482e-25,  7.38218882e-17,
         1.52892117e-10,  4.56134538e-07, -9.68997173e-06,
         9.64590465e-04, -7.76121954e-03, -1.38006335e-02,
         3.32601774e-03,  2.04

In [25]:
for m in M_list:
  try:
    np.linalg.inv(m[:,:,8,8])
  except np.linalg.LinAlgError as e:
    print(e)

Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix


In [27]:
np.linalg.inv(M_list[0][:,:,1,1])

LinAlgError: Singular matrix

In [33]:
eig = np.linalg.eig(M[:,:,0,0])

In [34]:
eig

EigResult(eigenvalues=array([ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.0000

In [36]:
svd = np.linalg.svd(M)

In [39]:
svd.u

AttributeError: 'SVDResult' object has no attribute 'u'